In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, avg, coalesce

spark = SparkSession.builder.getOrCreate()

In [2]:
df = spark.read.options(header=True).csv('s3://covid-data123/COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv')

Py4JJavaError: An error occurred while calling o24.csv.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "s3"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3281)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3301)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:124)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3352)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3320)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:479)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:361)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:46)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:376)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:326)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$3(DataFrameReader.scala:308)
	at scala.Option

In [3]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

22507139

In [13]:
df = df.filter((col('res_state') != 'NA') & (col('res_county') != 'NA'))
# df.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

20891650

In [14]:
df.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------+---------------+----------+----------------+---------+---+----+---------+-------------------------------+-------------------+-------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|case_month|res_state|state_fips_code|res_county|county_fips_code|age_group|sex|race|ethnicity|case_positive_specimen_interval|case_onset_interval|process|exposure_yn|      current_status|symptom_status|hosp_yn| icu_yn|death_yn|underlying_conditions_yn|
+----------+---------+---------------+----------+----------------+---------+---+----+---------+-------------------------------+-------------------+-------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|   2020-07|       SC|             45| ABBEVILLE|           45001|       NA| NA|  NA|       NA|                           null|                  0|Missing|    Missing|Laboratory-confir...|   Symptomatic|     No|Missing|      NA|          

In [17]:
filtered = df.filter((col('age_group') != 'NA') & (col('race') != 'NA') & (col('sex') != 'NA') & (col('ethnicity') != 'NA'))
filtered.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

16602569

In [20]:
cleaned = filtered.filter((col('age_group') != 'Missing') & (col('race') != 'Missing') & (col('sex') != 'Missing') & (col('ethnicity') != 'Missing'))
cleaned = cleaned.filter((col('age_group') != 'Unknown') & (col('race') != 'Unknown') & (col('sex') != 'Unknown') & (col('ethnicity') != 'Unknown'))
cleaned.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------+---------------+----------+----------------+--------------+------+--------------------+---------------+-------------------------------+-------------------+-------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|case_month|res_state|state_fips_code|res_county|county_fips_code|     age_group|   sex|                race|      ethnicity|case_positive_specimen_interval|case_onset_interval|process|exposure_yn|      current_status|symptom_status|hosp_yn| icu_yn|death_yn|underlying_conditions_yn|
+----------+---------+---------------+----------+----------------+--------------+------+--------------------+---------------+-------------------------------+-------------------+-------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|   2020-07|       AZ|             04|  MARICOPA|           04013|  0 - 17 years|Female|American Indian/A...|Hispanic/Latino|                       

In [21]:
cleaned.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

9132637

In [22]:
df = cleaned.select(col('res_state'), col('res_county'), col('age_group'), col('sex'), col('race'), col('ethnicity'))
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----------+--------------+------+--------------------+---------------+
|res_state|res_county|     age_group|   sex|                race|      ethnicity|
+---------+----------+--------------+------+--------------------+---------------+
|       AZ|  MARICOPA|  0 - 17 years|Female|American Indian/A...|Hispanic/Latino|
|       AZ|  MARICOPA|  0 - 17 years|Female|American Indian/A...|Hispanic/Latino|
|       AZ|  MARICOPA|  0 - 17 years|Female|American Indian/A...|Hispanic/Latino|
|       AZ|  MARICOPA|  0 - 17 years|Female|American Indian/A...|Hispanic/Latino|
|       AZ|      PIMA|  0 - 17 years|Female|American Indian/A...|Hispanic/Latino|
|       AZ|      PIMA|  0 - 17 years|Female|American Indian/A...|Hispanic/Latino|
|       AZ|      PIMA|  0 - 17 years|Female|American Indian/A...|Hispanic/Latino|
|       AZ|      PIMA|  0 - 17 years|Female|American Indian/A...|Hispanic/Latino|
|       NY|     BRONX|18 to 49 years|Female|American Indian/A...|Hispanic/Latino|
|       IL|     

In [23]:
cases = df.groupBy(col('res_county')).count()
cases.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-----+
|    res_county|count|
+--------------+-----+
|       JOHNSON|54843|
|         OSAGE| 2418|
|      PONTOTOC| 3118|
|     LA CROSSE| 8988|
|      BEAUFORT| 9888|
|        BRAZOS| 2362|
|        WALKER| 4232|
|     CLARENDON| 1219|
|    ORANGEBURG| 5744|
|        RACINE|15779|
|    GEORGETOWN| 4237|
|        PIERCE|12633|
|       MCCLAIN| 2894|
|        GIBSON| 4062|
|ST. LOUIS CITY| 3117|
|    BERNALILLO| 6496|
|      MONTEREY| 3116|
|         JONES| 2699|
|      CANADIAN| 9167|
|    GREENVILLE|38285|
+--------------+-----+
only showing top 20 rows

In [24]:
cases.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

989

In [26]:
states = df.groupBy(col('res_state')).count()
states.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------+
|res_state| count|
+---------+------+
|       MN|286652|
|       MD| 27611|
|       MO| 24493|
|       IL|511228|
|       MS| 10222|
|       SD| 27605|
|       AK| 12371|
|       UT|285111|
|       HI| 11454|
|       CA|850593|
|       CT| 60589|
|       NC|436223|
|       ME| 25650|
|       NM| 19984|
|       IA|132655|
|       AR|141352|
|       AZ|503947|
|       LA|  1045|
|       NJ|231757|
|       WI|354345|
+---------+------+
only showing top 20 rows

In [27]:
states.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

46

In [37]:
states.coalesce(1).write.csv('s3://covid-data123/processed_data/cases_by_states.csv')
cases.coalesce(1).write.csv('s3://covid-data123/processed_data/cases_by_counties.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
state_dem = df.select(col('res_state'), col('race'), col('sex'), col('age_group'), col('ethnicity'))
state_dem.coalesce(1).write.csv('s3://covid-data123/processed_data/state_demographics.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
month_case = cleaned.groupBy('case_month').count()
month_case.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+
|case_month|  count|
+----------+-------+
|   2020-05| 210942|
|   2021-02| 699414|
|   2020-11|1465315|
|   2020-07| 513753|
|   2020-03| 136208|
|   2020-09| 437849|
|   2020-06| 316126|
|   2021-03| 311304|
|   2020-01|    159|
|   2020-02|    169|
|   2020-08| 404229|
|   2021-01|1687891|
|   2020-10| 787475|
|   2020-12|1889466|
|   2020-04| 272337|
+----------+-------+

In [45]:
month_case.coalesce(1).write.csv('s3://covid-data123/processed_data/cases_by_month.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…